# Cryptocurrency Data Analysis

### Authors

|    Student Name                 |    Student Number  |
|---------------------------------|--------------------|
| Raj Sandhu                      | 101111960          |
| Akaash Kapoor                   | 101112895          |
| Ali Alvi                        | 101114940          |
| Hassan Jallad                   | 101109334          |
| Areeb Ul Haq                    | 101115337          |
| Ahmad Abuoudeh                  | 101072636          |

## Libraries to Import

In [1]:
! pip install gensim
! pip install pyemd
import gensim.downloader as api
import os
import pandas as pd
import numpy as np

## Get Path of Processed CSV File For Analysis

In [2]:
parent_folder = os.path.dirname(os.path.dirname(os.getcwd())) #Determine current working directory.
#Get the path of the processed coin info csv file from which analysis will be performed.
data_parent_folder = "data"
processed_data_folder = "processed"
processed_data_file = "coin-info.csv"
processed_data_file_path = os.path.join(parent_folder, data_parent_folder, processed_data_folder, processed_data_file) #Join path together.

## Get the Required Data of Two Test Coins

In [3]:
#Open the processed coin info csv file and obtain the descriptions of two test coins, whose cosine similarity shall be computed.
coin_info = pd.read_csv(processed_data_file_path)
first_coin_description = coin_info.loc[0]["Description"]
first_coin_volatility = coin_info.loc[0]["Volatility"]
first_coin_name = coin_info.loc[0]["Name"]
second_coin_description = coin_info.loc[1]["Description"]
second_coin_volatility = coin_info.loc[1]["Volatility"]
second_coin_name = coin_info.loc[1]["Name"]
print("The similarity of coins: " + first_coin_name + " and " + second_coin_name + " is to be computed by description.") #Output names of the two coins.
print("The similarity of coins: " + first_coin_name + " and " + second_coin_name + " is to be computed by volatility.") #Output names of the two coins.

The similarity of coins: iota and anchor-protocol is to be computed by description.
The similarity of coins: iota and anchor-protocol is to be computed by volatility.


## Load Pretrained model of Word Embeddings

In [4]:
#Call the pretrained word embedding model. This is an example of transfer learning.
model = api.load("word2vec-google-news-300")

## Apply the Model to the Descriptions of the Two Test Coins to Compute the Word Mover's Distance

In [5]:
word_movers_distance = model.wmdistance(first_coin_description, second_coin_description) #Apply pretrained word embedding model on descriptions.
print("The semantic similarity of the descriptions of " + first_coin_name + " and " + second_coin_name + " is " + str(word_movers_distance))

The semantic similarity of the descriptions of iota and anchor-protocol is 0.3376513362407437


## Compute the Euclidean Distance of the Market Volatility of the Two Test Coins

In [6]:
distance_volatility = np.linalg.norm(first_coin_volatility - second_coin_volatility) #Call the euclidean distance function from numpy.
print("The distance of the market volatilities of " + first_coin_name + " and " + second_coin_name + " is " + str(distance_volatility))

The distance of the market volatilities of iota and anchor-protocol is 0.7667485084340597


In the above two cells, tests were performed to check the similarity of the two test coins. The first was by using Word Mover's Distance on the coin descriptions and the second was by using the Euclidean distance of the market volatilities. The Euclidean distance of the two volatilities in 1-D is computed by the formula $\ |volatility_1 - volatility_2| $. Note that this is just exploring the analytical methods and shall be further refined in phase 3. However, these preliminary results appear promising and show good levels of similarity between the test coins.